In [8]:
import pandas as pd

# Import data

In [9]:
df_client=pd.read_csv("./raw_data/client.csv")
df_electricity=pd.read_csv("./raw_data/electricity_prices.csv")
df_gas=pd.read_csv("./raw_data/gas_prices.csv")

# Changing data format

### Manipulating dates
Check the number of unique values for the date column in the electricity
By checking only the unique dates, it will reduce the number of calculation when we will compare the date between the electricity df and the train df

In [10]:
print("Number of unique values in elec :", df_electricity["origin_date"].nunique())
print("Number of unique values in gas :", df_gas["origin_date"].nunique())

Number of unique values in elec : 15286
Number of unique values in gas : 637


In [11]:
# # Creating a new column "date" with the French formated date 
# # For elec
# df_electricity["date"] = pd.to_datetime(df_electricity["forecast_date"])
# df_electricity["date"] = df_electricity["date"].apply(lambda x : x.strftime("%y-%m-%d"))
# print(df_electricity["date"])
# 
# # For gases
# df_gas["date"] = pd.to_datetime(df_gas["forecast_date"])
# df_gas["date"] = df_gas["date"].apply(lambda x : x.strftime("%y-%m-%d"))
# print(df_gas["date"])

# Feature Engineering

### Adding prices in df_train
First of all we can see that the dataframe for the gas, do not use the same date format than the electricity and the train dataframes date format.
So we will be able to merge the info from the electricity dataframe, based on the datetime from the train dataframe.
But for the gas dataframe, we will need to add a formated column in train df. A column with the same format as the date in the gas df. We keep the date, we just remove the time.
After this manipulation, we can merge the gas prices too.

In [21]:
# Import train data
df_train = pd.read_csv("./raw_data/train.csv").sample(10000)

## Merge elec prices
# Creating a new column based on datetime
df_train["origin_date"] = df_train["datetime"]
# Selecting the infos we need to merge in the electricity df
df_elec_prices = df_electricity.filter(['euros_per_mwh', 'origin_date'], axis=1)
# Merging
df_train = df_train.merge(df_elec_prices, how="left", on="origin_date")

## Merge gas prices
# Removing the hours in the origin_date column created above
df_train["origin_date"] = df_train["origin_date"].apply(lambda x: pd.to_datetime(x).strftime("%Y-%m-%d"))
# Selecting the infos we need to merge in the gas df
df_gas_prices = df_gas.filter(['lowest_price_per_mwh', 'highest_price_per_mwh', 'origin_date'])
# Merging
df_train = df_train.merge(df_gas_prices, how="left", on="origin_date")

# Remove the origin_date column (because it's just a compacted version of "datetime")
df_train.drop(columns=["origin_date"], inplace=True)
# df_train.describe()
df_train

,county,is_business,product_type,target,is_consumption,datetime,data_block_id,row_id,prediction_unit_id,euros_per_mwh,lowest_price_per_mwh,highest_price_per_mwh
0,14,1,3,180.511,0,2021-09-17 15:00:00,16,48790,56,110.09,47.09,48.62
1,7,0,1,43.490,1,2021-10-22 20:00:00,51,153919,25,154.89,67.01,68.78
2,10,0,1,1.823,0,2021-11-09 15:00:00,69,207746,38,65.09,65.00,75.99
3,9,1,1,61.196,1,2022-01-02 19:00:00,123,371717,36,117.64,75.00,90.00
4,1,0,3,214.931,0,2022-05-10 09:00:00,251,778486,7,183.91,98.00,104.12
...,...,...,...,...,...,...,...,...,...,...,...,...
9995,5,1,1,168.353,0,2023-04-15 10:00:00,591,1870136,22,70.36,46.50,60.00
9996,5,1,0,40.860,1,2022-07-23 03:00:00,325,1014639,21,145.66,159.74,200.00
9997,9,0,1,22.459,1,2022-09-03 16:00:00,367,1149565,34,204.12,219.99,235.98
9998,2,1,1,37.480,1,2022-08-08 06:00:00,341,1065697,65,382.92,184.00,207.00


## Split datetime in multiple columns
Split into 4 columns the datetime (year, month, day, hour)

In [26]:
df_train["year"] = df_train["datetime"].apply(lambda x : x[0:4])
df_train["month"] = df_train["datetime"].apply(lambda x : x[5:7])
df_train["day"] = df_train["datetime"].apply(lambda x : x[8:10])
df_train["hour"] = df_train["datetime"].apply(lambda x : x[11:13])
df_train.drop(columns="datetime", inplace=True)

In [27]:
df_train.to_csv("./formated_data/train.csv",index=False)